In [1]:
import numpy as np
from cpmpy import *

import timeit
from IPython.display import display, HTML # display some titles
import random

In [ ]:
def OCUS(soft, soft_weights, hard=[], solver='ortools', verbose=1):
    ## Mip model
    if Model(hard+soft).solve():
        if verbose:
            print("Unexpectedly, the model is SAT")
        return []

    # decision variables of hitting set problem
    hs_vars = boolvar(shape=len(soft), name="hs_vars")

    hs_model = Model(
        # Objective: min sum(w_l * x_l)
        minimize=sum(soft_weights[id] * var for id, var in enumerate(hs_vars))
    )

    # instantiate hitting set solver
    hittingset_solver = SolverLookup.lookup(solver)(hs_model)

    while(True):
        # Get hitting set
        hittingset_solver.solve()
        hs_ids = np.where(hs_vars.value() == 1)[0]

        hs_soft = [soft[i] for i in hs_ids]

        # instantiate model every loop
        if not Model(hard+hs_soft).solve():
            return hs_soft

        # compute complement of model in formula F
        C = hs_vars[hs_vars.value() != 1]

        # Add complement as a new set to hit: sum x[j] * hij >= 1
        hittingset_solver += (sum(C) >= 1)

In [ ]:
def OCUS_assum(soft, soft_weights, hard=[], solver='ortools', verbose=1):
    # init with hard constraints
    assum_model = Model(hard)

    # make assumption indicators, add reified constraints
    ind = BoolVar(shape=len(soft), name="ind")
    for i,bv in enumerate(ind):
        assum_model += [bv.implies(soft[i])]
    # to map indicator variable back to soft_constraints
    indmap = dict((v,i) for (i,v) in enumerate(ind))

    assum_solver = SolverLookup.lookup(solver)(assum_model)

    if assum_solver.solve(assumptions=ind):
        return []

    ## 
    hs_model = Model(
        # Objective: min sum(x_l * w_l)
        minimize=sum(x_l * w_l for x_l, w_l in zip(ind, soft_weights))
    )

    # instantiate hitting set solver
    hittingset_solver = SolverLookup.lookup(solver)(hs_model)

    while(True):
        hittingset_solver.solve()

        # Get hitting set
        hs = ind[ind.value() == 1]


        if not assum_solver.solve(assumptions=hs):
            return soft[ind.value() == 1]

        # compute complement of model in formula F
        C = ind[ind.value() != 1]

        # Add complement as a new set to hit: sum x[j] * hij >= 1
        hittingset_solver += (sum(C) >= 1)

In [ ]:
def print_sudoku(grid):
    # Puzzel dimensions
    nrow = ncol = len(grid)
    n = int(nrow ** (1/2))

    out = ""    
    for r in range(0,nrow):
        for c in range(0,ncol):
            out += str(grid[r, c] if grid[r, c] > 0 else ' ')
            out += '  ' if grid[r, c] else '  '
            if (c+1) % n == 0 and c != nrow-1: # end of block
                out += '| '
        out += '\n'
        if (r+1) % n == 0 and r != nrow-1: # end of block
            out += ('-'*(n + 2*n))
            out += ('+' + '-'*(n + 2*n + 1)) *(n-1) + '\n'
    print(out)   

def model_sudoku(given):
    #empty cell
    e = 0
    # Dimensions of the sudoku problem
    ncol = nrow = len(given)
    n = int(ncol ** (1/2))

    # Model Variables
    cells = intvar(1,ncol,shape=given.shape,name="cells")
    
    # sudoku must start from initial clues
    facts = list(cells[given != e] == given[given != e])
    # rows constraint
    row_cons = [AllDifferent(row) for row in cells]
    # columns constraint
    col_cons = [AllDifferent(col) for col in cells.T]
    # blocks constraint
    # Constraints on blocks
    block_cons = []

    for i in range(0,nrow, n):
        for j in range(0,ncol, n):
            block_cons += [AllDifferent(cells[i:i+n, j:j+n])]

    return cells, facts, row_cons + col_cons + block_cons

def extract_solution(constraints, variables):
    solved = Model(constraints).solve()
    assert solved, "Model is unsatisfiable."     
    return variables.value()


In [ ]:
def unsat_int_model():
    weights = [10, 10, 10, 1, 1, 40, 20, 20, 20, 1]
    x = intvar(-9, 9, name="x")
    y = intvar(-9, 9, name="y")
    m = Model(
        x < 0, 
        x < 1,
        x > 2,
        x == 4,
        y == 4, 
        (x + y > 0) | (y < 0),
        (y >= 0) | (x >= 0),
        (y < 0) | (x < 0),
        (y > 0) | (x < 0),
        AllDifferent(x,y) # invalid for musx_assum
    )
    assert (m.solve() is False)

    print("\nStart OMUS search:")
    # run same code 5 times to get measurable data
    n = 100

    # calculate total execution time
    result_ocus = timeit.timeit(lambda: OCUS(soft=m.constraints, soft_weights=weights, hard=[]), number=n)
    print(f"OCUS without assumptions:\n\tExecution time is {result / n} seconds")
    
    
    result_ocus_assum = timeit.timeit(lambda: OCUS_assum(soft=np.array(m.constraints), soft_weights=weights, hard=[]), number=n)
    print(f"OCUS with assumptions:\n\tExecution time is {result / n} seconds")
    
    
    


In [ ]:
unsat_int_model()

In [ ]:
def unsat_sudoku_model():
    e = 0
    
    given_4x4 = np.array([
        [ e, 3, 4, e ],
        [ 4, e, e, 2 ],
        [ 1, e, e, 3 ],
        [ e, 2, 1, e ],
    ])
        
    given = given_4x4

    display(HTML('<h3> INPUT SUDOKU</h3>'))
    print_sudoku(given)
    
    sudoku_vars, sudoku_facts, sudoku_constraints = model_sudoku(given)

    sudoku_sol = extract_solution(
        constraints=sudoku_constraints + sudoku_facts, 
        variables=sudoku_vars
    )
    all_cons = sudoku_constraints + list(sudoku_facts)
    weights = [20] * len(sudoku_constraints) + [1] * (len(sudoku_facts) + 1)
    
    unassigned_negated = [var != sol_val for val, sol_val, var in zip(given.flatten(), sudoku_sol.flatten(), sudoku_vars.flatten()) if val == e]
    
    n = len(unassigned_negated)
    
    iter_ocus_negated = iter(unassigned_negated)
    
    result = timeit.timeit(lambda: OCUS(soft=all_cons + [next(iter_ocus_negated)], soft_weights=weights, hard=[]), number=1)
    print(f"OCUS without assumptions:\n\tExecution time is {result / n} seconds")
    
    iter_ocus_ass_negated = iter(unassigned_negated)
    result = timeit.timeit(lambda: OCUS_assum(soft=np.array(all_cons  + [next(iter_ocus_ass_negated)]), soft_weights=weights, hard=[]), number=1)
    print(f"OCUS with assumptions:\n\tExecution time is {result / n} seconds")

unsat_sudoku_model()